In [ ]:
# [Student: Merhawit, Student: Your Name 2, etc.] Import necessary libraries
import pandas as pd
import numpy as np
import random

# --- Part B: Technical Tasks ---

# --- 1. Simulate Transaction Data (Revised) ---

# [Student: Your Name] Define the path to the Groceries dataset CSV file
# Ensure 'Groceries_dataset.csv' is in the same directory as your script/notebook.
data_path = 'Groceries_dataset.csv'

# [Student: Your Name] Load the dataset into a pandas DataFrame
# The dataset has columns: 'Member_number', 'Date', 'itemDescription'.
try:
    df_raw = pd.read_csv(data_path)
    # [Student: Your Name] Display the first few rows and info of the raw data to confirm its structure
    print("--- Raw Groceries Data Head ---")
    print(df_raw.head())
    print("\n--- Raw Groceries Data Info ---")
    df_raw.info()
except FileNotFoundError:
    print(f"Error: The file '{data_path}' was not found.")
    print("Please download 'Groceries_dataset.csv' from https://www.kaggle.com/datasets/balajikartheek/groceries-dataset")
    print("and place it in the correct directory.")
    # [Student: Your Name] Exit the script if the file is not found
    exit()

--- Raw Groceries Data Head ---
   Member_number        Date   itemDescription
0           1808  21-07-2015    tropical fruit
1           2552  05-01-2015        whole milk
2           2300  19-09-2015         pip fruit
3           1187  12-12-2015  other vegetables
4           3037  01-02-2015        whole milk

--- Raw Groceries Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB
